# How to query for astrophysical data analysis

So far I have used
- Astropy
- Astroquery
- [PyVO](https://pyvo.readthedocs.io/en/latest/)



In [5]:
from astropy import table
from astropy.coordinates import SkyCoord
from pyvo import registry


my_obj = SkyCoord.from_name("Bellatrix")
for res in registry.search(waveband="infrared", servicetype="spectrum"):
    print(res.service.search(pos=my_obj, size=0.001))


<DALResultsTable length=4>
ssa_score location_ra location_dec ... instrument     telescope      expval
              deg         deg      ...                                     
 float32    float32     float32    ...   object         object      float64
--------- ----------- ------------ ... ---------- ----------------- -------
      0.0    81.28291    6.3496943 ...   COUDE700 Ondrejov Perek 2m 107.366
      0.0    81.28291    6.3496943 ...   COUDE700 Ondrejov Perek 2m 107.366
      0.0    81.28291    6.3496943 ...   COUDE700 Ondrejov Perek 2m  92.277
      0.0    81.28291    6.3496943 ...   COUDE700 Ondrejov Perek 2m  92.277
<DALResultsTable length=0>
ssa_score location_ra location_dec ... ssa_specend ssa_length ssa_fluxcalib
              deg         deg      ...      m                              
 float32    float32     float32    ...   float32     int32        object   
--------- ----------- ------------ ... ----------- ---------- -------------
<DALResultsTable length=12>
access

In [ ]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive


def query_multi_star_systems():
    # Query the database for systems with more than one star
    query_result = NasaExoplanetArchive.query_criteria(table="ps", select="*", where="sy_snum > 1")

    # Check if there are any results
    if len(query_result) == 0:
        print("No multi-star systems found.")
        return

    # Get the column names and print them
    column_names = query_result.columns
    print("Column Headers:")
    for column_name in column_names:
        print(column_name)

if __name__ == "__main__":
    query_multi_star_systems()


In [ ]:
from astropy import table
from pyvo import registry

QUERY = "SELECT TOP 1 s_ra, s_dec from ivoa.obscore"

results = []
for svc_rec in registry.search(
          datamodel="obscore", servicetype="tap"):
      print("Querying {}".format(svc_rec.res_title))
      try:
          svc = svc_rec.get_service("tap")
          results.append(
              svc.run_sync(QUERY).to_table())
      except KeyboardInterrupt:
          # someone lost their patience with a service.  Query next.
          pass
      except Exception as msg:
          # some service is broken; you *should* complain, but
          print("  Broken: {} ({}).  Complain to {}.\n".format(
              svc_rec.ivoid, msg, svc_rec.get_contact()))

total_result = table.vstack(results)
print(total_result)